In [1]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

import pandas as pd
import numpy as np
import utility_functions2 as uf
import string
import itertools
import timeit
import scipy.spatial.distance as dist


In [2]:
%%memit
n, m = 3000, 3000
name_gen = map(''.join, itertools.islice(itertools.product('ABCDE',repeat=6), n))
attr_gen = map(''.join, itertools.islice(itertools.product('EDCBA',repeat=6), m))
names = list(name_gen)
attr = list(attr_gen)

peak memory: 93.17 MiB, increment: 0.30 MiB


In [3]:
%%memit
# data = np.triu(np.random.random((n, m)))
# data = (data + data.T) / 2
data = np.random.randint(2, size=(n, m))
data

peak memory: 108.49 MiB, increment: 15.29 MiB


In [4]:
%%memit
df = pd.DataFrame(data, index=names, columns=attr)
df

peak memory: 101.09 MiB, increment: 0.12 MiB


In [5]:
%%memit 
similarity_matrix = dist.pdist(df, 'cosine')

peak memory: 112.52 MiB, increment: 11.39 MiB


In [6]:
%%memit 
similarity_matrix = dist.squareform(similarity_matrix)

peak memory: 112.56 MiB, increment: 0.04 MiB


In [7]:
%%memit 
similarity_matrix = 1 - similarity_matrix

peak memory: 120.19 MiB, increment: 7.63 MiB


In [8]:
%%memit 
similarity_df = pd.DataFrame(
    data=similarity_matrix, index=df.index, columns=df.index)

peak memory: 120.20 MiB, increment: 0.00 MiB


In [6]:
#%time uf.saveData(df, 'Output/', 'similarity_triu_savez_compressed', compression='npz', symmetric=False, dtype=np.float32)

In [8]:
#%%time
#df2 = uf.loadData('Output/similarity_triu_savez_compressed_2020_06.npz')
#df2

In [9]:
#assert(np.allclose(df, df2))

In [10]:
#%time uf.saveData(df, 'Output/', 'similarity_tsv', compression='gzip')